In [1]:
!pip install pyLDAvis==3.2.2

In [2]:
from sodapy import Socrata
import seaborn as sns 
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [7]:
from sodapy import Socrata
# get api token
client = Socrata("data.melbourne.vic.gov.au", #domain
                'z99BiHe97JrarLpbpqRISffyr',  #app token
                username="4l7fysec6unzmqbs1n9ulbdsz", #api id
                password="21kqwm0898v3yjkskdd2i940fc0g7quvr6rg80zalcyuhc1v4n")

## First step obtain datasets: 
  - dataset name 
  - id 
  - metadata

In [6]:
content = []
for a in client.datasets():

  content.append([a['resource']['name'],

  a['resource']['columns_name'],
  a['resource']['description'],
  a['classification']['categories'],
  a['classification']['domain_category'],
  a['classification']['tags'],
  a['classification']['domain_tags']])



# remove '[]' and join string
content_lst_inital = []
for each_element in content:
  each_element_list = []
  for element in each_element:
    if type(element) == list:
      element = ' '.join(element)
    else:
      pass
    each_element_list.append(element)
  cont= (' ').join(each_element_list)
  content_lst_inital.append(cont)

HTTPError: ignored

In [ ]:
content = []
for a in client.datasets():

  content.append([a['resource']['name'],
  a['resource']['columns_name'],
  a['resource']['description'],
  a['classification']['categories'],
  a['classification']['domain_category'],
  a['classification']['tags'],
  a['classification']['domain_tags']])

In [ ]:
total_list = []
for each_sent in content:
  test = []
  for i in each_sent:
    if type(i) == list:
      i = " ".join(i)
      test.append(i)
    else:
      test.append(i)
  line_list = " ".join(test)
  total_list.append(line_list)

In [ ]:
re.sub(r'[\(\)]',"", '(counts per ) (. ) ( ) (')

In [ ]:
# remove emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in total_list]

# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]

# html tags
data = [re.sub(r"<.*?> ", "", sent) for sent in data]

#
data = [re.sub(r'[^a-zA-Z]+', " ", sent) for sent in data]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

In [ ]:
!pip install importlib_metadata

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
data_lemmatized[0:8]

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

## Meta Data cleaning Done

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

## Table

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
df.head(5)

In [ ]:
!pip install nltk
import nltk
nltk.download('wordnet')
# Install spaCy (run in terminal/prompt)
import sys
!{sys.executable} -m pip install spacy

# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en

In [ ]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()





In [ ]:
each_char = 'trees'

lemmatizer.lemmatize(each_char, get_wordnet_pos(each_char))

In [ ]:
nlp_content = []
for each_sentence in words:
  list_1 = []
  for each_char in nltk.word_tokenize(each_sentence):
    tk=lemmatizer.lemmatize(each_char, get_wordnet_pos(each_char))
    list_1.append(tk)
  nlp_content.append(list_1)

In [ ]:
nlp_content[0]

In [ ]:
## check sparsicity 

# make it to sentence 
# nlp_content_string= sentence 
# nlp_content = nlp_content_string.split(' ')
nlp_content_string = [' '.join(i) for i in nlp_content]


#remove built-in english stopwords, convert all words to lowercase, 
#and a word can contain numbers and alphabets of at least length 3 in order to be qualified as a word.
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )


In [ ]:
data_vectorized = vectorizer.fit_transform(nlp_content_string)
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

In [ ]:
data_vectorized.shape

In [ ]:
search_params = {'n_components': list(range(1,223)), 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

In [ ]:
test_model_14 =LatentDirichletAllocation(batch_size=6, n_components = 14,
                        random_state=0,
                        learning_decay=0.7,
                        max_iter= 10,
                        max_doc_update_iter=100,
                        learning_method='batch')
test_model_14.fit(data_vectorized)

In [ ]:
test_model_6 =LatentDirichletAllocation(batch_size=6, n_components = 6,
                        random_state=0,
                        learning_decay=0.7,
                        max_iter= 10,
                        max_doc_update_iter=100,
                        learning_method='batch')
test_model_6.fit(data_vectorized)

In [ ]:


#data is data_vectorized
def return_topic_visual_pro_topic(model, data=data_vectorized):

  # Create Document - Topic Matrix
  lda_output = model.transform(data)
  # column names
  topicnames = ["Topic" + str(i) for i in range(model.n_components)]
  # index names
  docnames = ["dataset" + str(i) for i in range(data.shape[0])]

  # Make the pandas dataframe
  df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

  # Get dominant topic for each document
  dominant_topic = np.argmax(df_document_topic.values, axis=1)
  df_document_topic['dominant_topic'] = dominant_topic

  # Styling
  def color_green(val):
      color = 'green' if val > .1 else 'black'
      return 'color: {col}'.format(col=color)

  def make_bold(val):
      weight = 700 if val > .1 else 400
      return 'font-weight: {weight}'.format(weight=weight)


  # Apply Style
  df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
  

  return  df_document_topics, dominant_topic


In [ ]:
# replace model to see difference 
df_document_topics, dominant_topic  = return_topic_visual_pro_topic(test_model_14, data_vectorized)
df_document_topics


In [ ]:
#topic_n = df_document_topic['dominant_topic']
#df.loc[:,'topic_n'] = topic_n.values
df['content'] = nlp_content_string
df['topic_n'] = dominant_topic

In [ ]:
df[df.loc[:,'topic_n'] == 13]

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(test_model_14, data_vectorized, vectorizer, mds='tsne')
panel